In [73]:
import numpy as np
import pandas as pd
import json
import math
import plotly.express as px
from area import area

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

In [205]:
df = pd.read_csv('Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv')
df = df[df['City'] == 'New York']

nycmap = json.load(open("nyc_neighborhoods.geojson"))

dates = ('2020-01-31',
         '2020-02-29',
         '2020-03-31',
         '2020-04-30',
         '2020-05-31',
         '2020-06-30',
         '2020-07-31',
         '2020-08-31',
         '2020-09-30',
         '2020-10-31')


In [75]:
df.loc[(df.CountyName == 'New York County'),'CountyName'] = 'Manhattan'
df.loc[(df.CountyName == 'Kings County'),'CountyName'] = 'Brooklyn'
df.loc[(df.CountyName == 'Queens County'),'CountyName'] = 'Queens'
df.loc[(df.CountyName == 'Richmond County'),'CountyName'] = 'Staten Island'
df.loc[(df.CountyName == 'Bronx County'),'CountyName'] = 'Bronx'

In [76]:
df.loc[(df['RegionName'] == 'Brooklyn Heights') & (df['CountyName'] == 'Manhattan'), 'CountyName'] = 'Brooklyn'
df.loc[(df['RegionName'] == 'Throggs Neck') & (df['CountyName'] == 'Bronx'), 'RegionName'] = 'Throgs Neck'
df.loc[(df['RegionName'] == 'Rochdale') & (df['CountyName'] == 'Queens'), 'RegionName'] = 'Springfield Gardens North'
df.loc[(df['RegionName'] == 'Springfield Gardens') & (df['CountyName'] == 'Queens'), 'RegionName'] = 'Springfield Gardens South'
df.loc[(df['RegionName'] == 'Financial District') & (df['CountyName'] == 'Manhattan'), 'RegionName'] = 'Lower Manhattan'
df.loc[(df['RegionName'] == 'Flatiron District') & (df['CountyName'] == 'Manhattan'), 'RegionName'] = 'Flatiron'
df.loc[(df['RegionName'] == 'West Brighton') & (df['CountyName'] == 'Staten Island'), 'RegionName'] = 'West New Brighton'
df.loc[(df['RegionName'] == 'Battery Park') & (df['CountyName'] == 'Manhattan'), 'RegionName'] = 'Battery Park City'
df.loc[(df['RegionName'] == 'Downtown') & (df['CountyName'] == 'Brooklyn'), 'RegionName'] = 'Downtown Brooklyn'
df.loc[(df['RegionName'] == 'Grasmere - Concord') & (df['CountyName'] == 'Staten Island'), 'RegionName'] = 'Grasmere'
df.loc[(df['RegionName'] == 'Columbia Street Waterfront District') & (df['CountyName'] == 'Brooklyn'), 'RegionName'] = 'Columbia Street'
df.loc[(df['RegionName'] == 'Fort Wadsworth') & (df['CountyName'] == 'Staten Island'), 'RegionName'] = 'Wadsworth'
df.loc[(df['RegionName'] == 'Chelsea-Travis') & (df['CountyName'] == 'Staten Island'), 'RegionName'] = 'Travis'
df.loc[(df['RegionName'] == 'Blissville') & (df['CountyName'] == 'Queens'), 'RegionName'] = 'West Maspeth'
df.loc[(df['RegionName'] == 'John F. Kennedy International Airport') & (df['CountyName'] == 'Queens'), 'RegionName'] = 'Airport'

In [77]:
temp_df = df[['RegionName', 'CountyName']]
neigh_boro = temp_df.values.tolist()

In [78]:
new_frame = pd.DataFrame(columns = ['ntaname', 'boro_name', 'count'])

for n in nycmap['features']:
    new_frame = new_frame.append({'ntaname': n["properties"]["ntaname"], 'boro_name': n["properties"]["boro_name"]}, ignore_index = True)

    
for d in dates:
    new_frame[d] = 0

    
new_frame["count"] = 0


In [79]:
def contains(whole, part):
    whole = whole.lower()
    part = part.lower()
    
    words = whole.split('-')
    
    for w in words:
        if part == w:
            return True
    
    return False

In [80]:
has_parts = [['Washington Heights', 'Manhattan'],
            ['Crown Heights', 'Brooklyn'],
            ['Harlem', 'Manhattan'],
            ['Bushwick', 'Brooklyn'],
            ['Sunset Park', 'Brooklyn'],
            ['Bensonhurst', 'Brooklyn'],
            ['Tremont', 'Bronx'],
            ['Fordham', 'Bronx'],
            ['Douglaston-Little Neck', 'Queens'],
            ['Co-op City', 'Bronx'],]

In [81]:
part_of = [
            (['Greenwich Village', 'Manhattan'], 'West Village'),
            (['Fort Hamilton', 'Brooklyn'], 'Bay Ridge'),
            (['Sutton Place', 'Manhattan'], 'East Midtown'),
            (['New Utrecht', 'Brooklyn'],  'Bensonhurst West'),
            (['Greenwood', 'Brooklyn'], 'Sunset Park West'),
            (['Westchester Heights', 'Bronx'], 'Westchester', 'Westchester Square', 'Westchester-Unionport'),
            (['Bulls Head', 'Staten Island'], 'New Springville'),
            (['Willowbrook', 'Staten Island'], 'New Springville'),
            (['Park Hill', 'Staten Island'], 'Clifton'),
            (['Greenridge', 'Staten Island'], 'Eltingville'),
            (['Richmond Town', 'Staten Island'], 'Oakwood'),
            (['Grant City', 'Staten Island'], 'Midland Beach'),
            (['Castleton Corners', 'Staten Island'], 'Westerleigh'),
            (['Bay Terrace', 'Staten Island'], 'Oakwood'),
            (['Meiers Corners', 'Staten Island'], 'Westerleigh'),
            (['Tompkinsville', 'Staten Island'], 'Stapleton'),
            (['NoHo', 'Manhattan'], 'West Village'),
            (['Astoria Heights', 'Queens'], 'Steinway'),
            (['Garment District', 'Manhattan'], 'Midtown South'),
            (['Forest Hills Gardens', 'Queens'], 'Forest Hills'),
            (['Pleasant Plains', 'Staten Island'], 'Charleston'),
            (['Sunnyside', 'Staten Island'], 'Westerleigh'),
            (['New Dorp Beach', 'Staten Island'], 'New Dorp'),
            (['Elm Park', 'Staten Island'], 'Port Richmond'),
            (['Prospect Park South', 'Brooklyn'], 'Flatbush'),
            (['Butler Manor', 'Staten Island'], 'Tottenville'),
            (['Malba', 'Queens'], 'Whitestone'),
            (['Pelham Bay Park', 'Bronx'], 'Pelham Bay'),               
            (['Neponsit', 'Queens'], 'Belle Harbor'),
            (['Shore Acres', 'Staten Island'], 'Rosebank'),
            (['Bedford Stuyvesant', 'Brooklyn'],'Bedford', 'Stuyvesant Heights')
]


def isPartOf(n):
    for a in part_of:
        if a.count(n) > 0:
            return a
        
    return []

In [82]:
new_frame.loc[new_frame['ntaname'] == 'park-cemetery-etc-Queens']

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
194,park-cemetery-etc-Queens,Queens,0,0,0,0,0,0,0,0,0,0,0


In [83]:
not_mapped = []

for n in neigh_boro:
    not_mapped.append(n)
    
    if has_parts.count(n) > 0:
        for i in range(len(new_frame.index)):
            row = new_frame.iloc[i:i+1, :]
            if n[0] in row.ntaname[i]:
                if row.boro_name[i] == n[1]:
                    curr_count = new_frame.iloc[i:i+1, 2]
                    new_frame.iloc[i:i+1, 2] = curr_count + 1
                    
                    for c in range(len(dates)):
                        curr_val = new_frame.iloc[i:i+1, len(new_frame.columns)-len(dates)+c]
                        curr_row = df.loc[(df['RegionName'] == n[0]) & (df['CountyName'] == n[1])]
                        addend = curr_row.iloc[0, len(curr_row.columns)-len(dates)+c]
                        new_frame.iloc[i:i+1, len(new_frame.columns)-len(dates)+c] = addend + curr_val
                    
        not_mapped.remove(n)
        
    elif len(isPartOf(n)) > 0:
        line = isPartOf(n)
        region = line[0][0]
        county = line[0][1]
        parts = line[1:]
        
        for p in parts:
            curr_count = new_frame.loc[(new_frame['ntaname'] == p) & (new_frame['boro_name'] == county), 'count']
            new_frame.loc[(new_frame['ntaname'] == p) & (new_frame['boro_name'] == county), 'count'] = curr_count + 1
            
            for c in range(len(dates)):
                curr_val = new_frame.loc[(new_frame['ntaname'] == p) & (new_frame['boro_name'] == county), new_frame.columns[len(new_frame.columns)-len(dates)+c]]
                curr_row = df.loc[(df['RegionName'] == n[0]) & (df['CountyName'] == n[1])]
                addend = curr_row.iloc[0, len(curr_row.columns)-len(dates)+c]
                new_frame.loc[(new_frame['ntaname'] == p) & (new_frame['boro_name'] == county), new_frame.columns[len(new_frame.columns)-len(dates)+c]] = addend + curr_val
            
        not_mapped.remove(n)
        
    else:
        for i in range(len(new_frame.index)):
            row = new_frame.iloc[i:i+1, :]
            if contains(row.ntaname[i], n[0]):
                if row.boro_name[i] == n[1]:
                    curr_count = new_frame.iloc[i:i+1, 2]
                    new_frame.iloc[i:i+1, 2] = curr_count + 1
                    
                    for c in range(len(dates)):
                        curr_val = new_frame.iloc[i:i+1, len(new_frame.columns)-len(dates)+c]
                        curr_row = df.loc[(df['RegionName'] == n[0]) & (df['CountyName'] == n[1])]
                        addend = curr_row.iloc[0, len(curr_row.columns)-len(dates)+c]
                        new_frame.iloc[i:i+1, len(new_frame.columns)-len(dates)+c] = addend + curr_val
                    
                    not_mapped.remove(n)
                    break

                    
for r in range(len(new_frame)):
    if new_frame.iloc[r, 2] > 1:
        for c in range(len(dates)):
            curr = new_frame.iloc[r, len(new_frame.columns)-len(dates)+c]
            count = new_frame.iloc[r, 2]
            new_frame.iloc[r, len(new_frame.columns)-len(dates)+c] = curr / count

In [84]:
new_frame

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
0,Borough Park,Brooklyn,1,800107.000000,794346.0,792962.000000,792549.000000,792589.000000,791801.000000,790573.000000,792553.0,794147.000000,792330.0
1,Murray Hill,Queens,1,384721.000000,384396.0,383443.000000,383618.000000,383362.000000,383960.000000,384397.000000,384940.0,386321.000000,388206.0
2,East Elmhurst,Queens,1,663453.000000,665648.0,667851.000000,669964.000000,671708.000000,672445.000000,672120.000000,672802.0,676621.000000,681728.0
3,Erasmus,Brooklyn,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,Hollis,Queens,1,554907.000000,555797.0,556189.000000,557034.000000,557172.000000,558160.000000,560169.000000,562641.0,566751.000000,568297.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,park-cemetery-etc-Brooklyn,Brooklyn,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
191,Richmond Hill,Queens,1,571840.000000,572333.0,572168.000000,572098.000000,571145.000000,572393.000000,574503.000000,577717.0,581482.000000,583228.0
192,Jamaica,Queens,1,439332.000000,440256.0,441169.000000,442618.000000,443684.000000,445890.000000,448625.000000,451519.0,455501.000000,458605.0
193,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,Queens,3,531565.333333,533002.0,533948.666667,536517.666667,537704.666667,539185.333333,539979.666667,541870.0,544999.666667,548942.0


In [85]:
new_frame.loc[new_frame['ntaname'] == 'Whitestone']

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
151,Whitestone,Queens,2,716150.5,714378.5,712513.5,712687.0,712395.5,714074.5,716273.0,719226.5,722598.0,725292.0


In [14]:
df.loc[df['RegionName'] == 'Prospect Lefferts Gardens']

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
337,403221,350,Prospect Lefferts Gardens,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,Brooklyn,116114.0,...,728109.0,722240.0,717533.0,716227.0,714956.0,716838.0,719266.0,724660.0,725096.0,725565.0


In [15]:
new_frame.loc[new_frame['count'] == 0]

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
3,Erasmus,Brooklyn,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,Starrett City,Brooklyn,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,Madison,Brooklyn,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,Rugby-Remsen Village,Brooklyn,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,Ocean Parkway South,Brooklyn,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,Queensbridge-Ravenswood-Long Island City,Queens,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,Van Cortlandt Village,Bronx,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,Clinton,Manhattan,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,East New York (Pennsylvania Ave),Brooklyn,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,Bronxdale,Bronx,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
new_frame.columns[4]

new_frame.columns[len(curr_row.columns)-len(dates)+c]

IndexError: index 306 is out of bounds for axis 0 with size 13

In [ ]:
#wrong borough, typo, 
'''['Brooklyn Heights', 'Manhattan'], #brooklyn
['Throggs Neck', 'Bronx'], #Throgs
['Rochdale', 'Queens'], Springfield Gardens North
['Springfield Gardens', 'Queens'], Springfield Gardens South
['Financial District', 'Manhattan'], Lower Manhattan
['Flatiron District', 'Manhattan'], Flatiron
['West Brighton', 'Staten Island'], West New Brighton
['Battery Park', 'Manhattan'], Battery Park City
['Downtown', 'Brooklyn'], Downtown Brooklyn
['Grasmere - Concord', 'Staten Island'], Grasmere
['Columbia Street Waterfront District', 'Brooklyn'], Columbia Street
['Fort Wadsworth', 'Staten Island'], Wadsworth
['Chelsea-Travis', 'Staten Island'], Travis
['Blissville', 'Queens'], West Maspeth
['John F. Kennedy International Airport', 'Queens'], Airport'''

In [ ]:
not_mapped

'''ignore
['South Bronx', 'Bronx'],
['Central Park', 'Manhattan'],
['Navy Yard', 'Brooklyn']]'''

In [ ]:
corr_list = [['Washington Heights', 'Manhattan'],
 ['Bedford Stuyvesant', 'Brooklyn'],
 ['Crown Heights', 'Brooklyn'],
 ['Harlem', 'Manhattan'],
 ['Bushwick', 'Brooklyn'],
 ['Sunset Park', 'Brooklyn'],
 ['Bensonhurst', 'Brooklyn'],
 ['South Bronx', 'Bronx'],
 ['Tremont', 'Bronx'],
 ['Fordham', 'Bronx'],
 ['Greenwich Village', 'Manhattan'],
 ['Co-op City', 'Bronx'],
 ['Throggs Neck', 'Bronx'],
 ['Rochdale', 'Queens'],
 ['Financial District', 'Manhattan'],
 ['Fort Hamilton', 'Brooklyn'],
 ['Flatiron District', 'Manhattan'],
 ['Douglaston-Little Neck', 'Queens'],
 ['Brooklyn Heights', 'Manhattan'],
 ['Sutton Place', 'Manhattan'],
 ['West Brighton', 'Staten Island'],
 ['Springfield Gardens', 'Queens'],
 ['New Utrecht', 'Brooklyn'],
 ['Greenwood', 'Brooklyn'],
 ['Battery Park', 'Manhattan'],
 ['Westchester Heights', 'Bronx'],
 ['Bulls Head', 'Staten Island'],
 ['Downtown', 'Brooklyn'],
 ['Willowbrook', 'Staten Island'],
 ['Park Hill', 'Staten Island'],
 ['Greenridge', 'Staten Island'],
 ['Richmond Town', 'Staten Island'],
 ['Grant City', 'Staten Island'],
 ['Castleton Corners', 'Staten Island'],
 ['Grasmere - Concord', 'Staten Island'],
 ['Bay Terrace', 'Staten Island'],
 ['Meiers Corners', 'Staten Island'],
 ['Tompkinsville', 'Staten Island'],
 ['NoHo', 'Manhattan'],
 ['Astoria Heights', 'Queens'],
 ['Garment District', 'Manhattan'],
 ['Columbia Street Waterfront District', 'Brooklyn'],
 ['Forest Hills Gardens', 'Queens'],
 ['Pleasant Plains', 'Staten Island'],
 ['Sunnyside', 'Staten Island'],
 ['New Dorp Beach', 'Staten Island'],
 ['Elm Park', 'Staten Island'],
 ['Prospect Park South', 'Brooklyn'],
 ['Butler Manor', 'Staten Island'],
 ['Chelsea-Travis', 'Staten Island'],
 ['Central Park', 'Manhattan'],
 ['Fort Wadsworth', 'Staten Island'],
 ['Malba', 'Queens'],
 ['Pelham Bay Park', 'Bronx'],
 ['Neponsit', 'Queens'],
 ['Shore Acres', 'Staten Island'],
 ['Blissville', 'Queens'],
 ['John F. Kennedy International Airport', 'Queens'],
 ['Navy Yard', 'Brooklyn']]

In [50]:
hover_data = [('Whitestone', 'Queens')]

In [51]:
 hover_data[0][0]

'Whitestone'

In [52]:
frame = new_frame.loc[(new_frame['ntaname'] == hover_data[0][0]) & (new_frame['boro_name'] == hover_data[0][1])]

frame

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
151,Whitestone,Queens,2,716150.5,714378.5,712513.5,712687.0,712395.5,714074.5,716273.0,719226.5,722598.0,725292.0


In [53]:
default_sf = new_frame.loc[(new_frame["ntaname"] == 'Hamilton Heights') & (new_frame["boro_name"] == 'Manhattan')]
tr = sf.iloc[:, len(default_sf.columns)-len(dates):].T
tr.reset_index(inplace=True)
tr.columns = ['date', 'price']
tr['ntaname'] = 'Hamilton Heights'
tr['boro_name'] = 'Manhattan'

scatter_fig = px.line(tr, x='date', y='price', color='ntaname')

scatter_fig.show()

In [54]:
selected_frame = pd.DataFrame()

for d in hover_data:
    sf = new_frame.loc[(new_frame['ntaname'] == d[0]) & (new_frame['boro_name'] == d[1])]
    transposed = sf.iloc[:, len(sf.columns)-len(dates):].T
    transposed.reset_index(inplace=True)
    transposed.columns = ['date', 'price']
    transposed['ntaname'] = d[0]
    transposed['boro_name'] = d[1]
    selected_frame = pd.concat([selected_frame, transposed])
    
selected_frame

,date,price,ntaname,boro_name
0,2020-01-31,716150.5,Whitestone,Queens
1,2020-02-29,714378.5,Whitestone,Queens
2,2020-03-31,712513.5,Whitestone,Queens
3,2020-04-30,712687.0,Whitestone,Queens
4,2020-05-31,712395.5,Whitestone,Queens
5,2020-06-30,714074.5,Whitestone,Queens
6,2020-07-31,716273.0,Whitestone,Queens
7,2020-08-31,719226.5,Whitestone,Queens
8,2020-09-30,722598.0,Whitestone,Queens
9,2020-10-31,725292.0,Whitestone,Queens


In [55]:
print(selected_frame)

         date     price     ntaname boro_name
0  2020-01-31  716150.5  Whitestone    Queens
1  2020-02-29  714378.5  Whitestone    Queens
2  2020-03-31  712513.5  Whitestone    Queens
3  2020-04-30  712687.0  Whitestone    Queens
4  2020-05-31  712395.5  Whitestone    Queens
5  2020-06-30  714074.5  Whitestone    Queens
6  2020-07-31  716273.0  Whitestone    Queens
7  2020-08-31  719226.5  Whitestone    Queens
8  2020-09-30  722598.0  Whitestone    Queens
9  2020-10-31  725292.0  Whitestone    Queens


In [57]:
scatter_fig = px.line(selected_frame,
                      x='date',
                      y='price',
                      color='ntaname'
                      )

scatter_fig.update_layout(width=1200,
                              height=400
                              )
scatter_fig.show()

In [58]:
selected_frame['price'].min()

712395.5

In [28]:
#hover_frame = new_frame.loc[new_frame['ntaname'].isin(hover_data)]
hover_frame1 = new_frame.loc[new_frame['ntaname'] == hover_data[0]]

hover_frame1

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31


In [130]:
hover_frame2 = new_frame.loc[new_frame['ntaname'] == hover_data[1]]

hover_frame2

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
1,Murray Hill,Queens,1,384721.0,384396.0,383443.0,383618.0,383362.0,383960.0,384397.0,384940.0,386321.0,388206.0


In [131]:
concat = pd.concat([hover_frame1, hover_frame2])

concat

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
151,Whitestone,Queens,2,716150.5,714378.5,712513.5,712687.0,712395.5,714074.5,716273.0,719226.5,722598.0,725292.0
1,Murray Hill,Queens,1,384721.0,384396.0,383443.0,383618.0,383362.0,383960.0,384397.0,384940.0,386321.0,388206.0


In [132]:
transposed = hover_frame1.iloc[:, len(hover_frame)-len(dates):].T

transposed

,151
2020-04-30,712687.0
2020-05-31,712395.5
2020-06-30,714074.5
2020-07-31,716273.0
2020-08-31,719226.5
2020-09-30,722598.0
2020-10-31,725292.0


In [133]:
transposed = hover_frame1.iloc[:, len(hover_frame)-len(dates):].T
transposed.reset_index(inplace=True)
transposed.columns = ['dates', 'prices']

transposed


,dates,prices
0,2020-04-30,712687.0
1,2020-05-31,712395.5
2,2020-06-30,714074.5
3,2020-07-31,716273.0
4,2020-08-31,719226.5
5,2020-09-30,722598.0
6,2020-10-31,725292.0


In [135]:
transposed['name'] = 'e'

transposed

,dates,prices,name
0,2020-04-30,712687.0,e
1,2020-05-31,712395.5,e
2,2020-06-30,714074.5,e
3,2020-07-31,716273.0,e
4,2020-08-31,719226.5,e
5,2020-09-30,722598.0,e
6,2020-10-31,725292.0,e


In [108]:
transposed.head()

,dates,prices
0,2020-02-29,714378.5
1,2020-03-31,712513.5
2,2020-04-30,712687.0
3,2020-05-31,712395.5
4,2020-06-30,714074.5


In [111]:
fig = px.line(transposed,
              x='dates',
              y='prices'
              )

fig.show()

In [109]:
transposed.columns

Index(['dates', 'prices'], dtype='object')

In [110]:
new_frame.columns

Index(['ntaname', 'boro_name', 'count', '2020-01-31', '2020-02-29',
       '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31',
       '2020-08-31', '2020-09-30', '2020-10-31'],
      dtype='object')

In [ ]:
, ('Murray Hill', 'Queens'), ('Richmond Hill', 'Queens')

In [59]:
new_frame

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
0,Borough Park,Brooklyn,1,800107.000000,794346.0,792962.000000,792549.000000,792589.000000,791801.000000,790573.000000,792553.0,794147.000000,792330.0
1,Murray Hill,Queens,1,384721.000000,384396.0,383443.000000,383618.000000,383362.000000,383960.000000,384397.000000,384940.0,386321.000000,388206.0
2,East Elmhurst,Queens,1,663453.000000,665648.0,667851.000000,669964.000000,671708.000000,672445.000000,672120.000000,672802.0,676621.000000,681728.0
3,Erasmus,Brooklyn,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
4,Hollis,Queens,1,554907.000000,555797.0,556189.000000,557034.000000,557172.000000,558160.000000,560169.000000,562641.0,566751.000000,568297.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,park-cemetery-etc-Brooklyn,Brooklyn,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
191,Richmond Hill,Queens,1,571840.000000,572333.0,572168.000000,572098.000000,571145.000000,572393.000000,574503.000000,577717.0,581482.000000,583228.0
192,Jamaica,Queens,1,439332.000000,440256.0,441169.000000,442618.000000,443684.000000,445890.000000,448625.000000,451519.0,455501.000000,458605.0
193,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,Queens,3,531565.333333,533002.0,533948.666667,536517.666667,537704.666667,539185.333333,539979.666667,541870.0,544999.666667,548942.0


In [60]:
new_frame.T

,0,1,2,3,4,5,6,7,8,9,...,185,186,187,188,189,190,191,192,193,194
ntaname,Borough Park,Murray Hill,East Elmhurst,Erasmus,Hollis,Homecrest,Fresh Meadows-Utopia,Longwood,Midwood,St. Albans,...,Airport,Cypress Hills-City Line,Ozone Park,East New York,Lindenwood-Howard Beach,park-cemetery-etc-Brooklyn,Richmond Hill,Jamaica,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,park-cemetery-etc-Queens
boro_name,Brooklyn,Queens,Queens,Brooklyn,Queens,Brooklyn,Queens,Bronx,Brooklyn,Queens,...,Queens,Brooklyn,Queens,Brooklyn,Queens,Brooklyn,Queens,Queens,Queens,Queens
count,1,1,1,0,1,1,2,1,1,1,...,1,0,1,1,1,0,1,1,3,0
2020-01-31,800107,384721,663453,0,554907,465258,899562,363632,618637,524472,...,624473,0,548951,469786,432327,0,571840,439332,531565,0
2020-02-29,794346,384396,665648,0,555797,461046,897967,369056,616965,525775,...,626760,0,549895,473683,434033,0,572333,440256,533002,0
2020-03-31,792962,383443,667851,0,556189,458023,893620,371127,615719,526581,...,625484,0,550105,477718,435862,0,572168,441169,533949,0
2020-04-30,792549,383618,669964,0,557034,455629,890875,374941,616133,527156,...,625442,0,550713,481273,437551,0,572098,442618,536518,0
2020-05-31,792589,383362,671708,0,557172,453952,886710,379046,616338,527504,...,626810,0,550456,484068,439011,0,571145,443684,537705,0
2020-06-30,791801,383960,672445,0,558160,449727,884524,384528,616457,528954,...,633311,0,551628,487288,440854,0,572393,445890,539185,0
2020-07-31,790573,384397,672120,0,560169,446202,880108,389711,615549,529913,...,638299,0,553789,490425,442839,0,574503,448625,539980,0


In [61]:
data = px.data.gapminder()

In [62]:
data

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


In [69]:
new_frame.loc[1, ['ntaname', 'boro_name', ]

SyntaxError: unexpected EOF while parsing (<ipython-input-69-f0d6f3965afb>, line 1)

In [230]:
dates_frame = pd.DataFrame()

for r in range(len(new_frame)):
    date_row = new_frame.iloc[r:r+1, :]
    curr_region = date_row.iloc[0, 0]
    curr_boro = date_row.iloc[0, 1]
    dates_transposed = date_row.iloc[:, len(date_row.columns) - len(dates):].T
    dates_transposed.reset_index(inplace=True)
    dates_transposed.columns = ['date', 'price']
    dates_transposed['ntaname'] = curr_region
    dates_transposed['boro_name'] = curr_boro
    dates_frame = pd.concat([dates_frame, dates_transposed]).reset_index(drop=True)
    
dates_frame

,date,price,ntaname,boro_name
0,2020-02-29,794346.0,Borough Park,Brooklyn
1,2020-03-31,792962.0,Borough Park,Brooklyn
2,2020-04-30,792549.0,Borough Park,Brooklyn
3,2020-05-31,792589.0,Borough Park,Brooklyn
4,2020-06-30,791801.0,Borough Park,Brooklyn
...,...,...,...,...
1945,2020-07-31,0.0,park-cemetery-etc-Queens,Queens
1946,2020-08-31,0.0,park-cemetery-etc-Queens,Queens
1947,2020-09-30,0.0,park-cemetery-etc-Queens,Queens
1948,2020-10-31,0.0,park-cemetery-etc-Queens,Queens


In [231]:
f = dates_frame.loc[dates_frame['date'] == '2020-03-31']
s = dates_frame.loc[dates_frame['date'] == '2020-08-31']


In [232]:
f

,date,price,ntaname,boro_name
1,2020-03-31,792962.000000,Borough Park,Brooklyn
11,2020-03-31,383443.000000,Murray Hill,Queens
21,2020-03-31,667851.000000,East Elmhurst,Queens
31,2020-03-31,0.000000,Erasmus,Brooklyn
41,2020-03-31,556189.000000,Hollis,Queens
...,...,...,...,...
1901,2020-03-31,0.000000,park-cemetery-etc-Brooklyn,Brooklyn
1911,2020-03-31,572168.000000,Richmond Hill,Queens
1921,2020-03-31,441169.000000,Jamaica,Queens
1931,2020-03-31,533948.666667,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,Queens


In [233]:
s

,date,price,ntaname,boro_name
6,2020-08-31,792553.0,Borough Park,Brooklyn
16,2020-08-31,384940.0,Murray Hill,Queens
26,2020-08-31,672802.0,East Elmhurst,Queens
36,2020-08-31,0.0,Erasmus,Brooklyn
46,2020-08-31,562641.0,Hollis,Queens
...,...,...,...,...
1906,2020-08-31,0.0,park-cemetery-etc-Brooklyn,Brooklyn
1916,2020-08-31,577717.0,Richmond Hill,Queens
1926,2020-08-31,451519.0,Jamaica,Queens
1936,2020-08-31,541870.0,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,Queens


In [237]:
new = dates_frame.loc[dates_frame['date'] == '2020-08-31']

new['new'] = f['price'].values-s['price'].values

new

C:\Users\Sena\.conda\envs\prog\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,price,ntaname,boro_name,new
6,2020-08-31,792553.0,Borough Park,Brooklyn,409.000000
16,2020-08-31,384940.0,Murray Hill,Queens,-1497.000000
26,2020-08-31,672802.0,East Elmhurst,Queens,-4951.000000
36,2020-08-31,0.0,Erasmus,Brooklyn,0.000000
46,2020-08-31,562641.0,Hollis,Queens,-6452.000000
...,...,...,...,...,...
1906,2020-08-31,0.0,park-cemetery-etc-Brooklyn,Brooklyn,0.000000
1916,2020-08-31,577717.0,Richmond Hill,Queens,-5549.000000
1926,2020-08-31,451519.0,Jamaica,Queens,-10350.000000
1936,2020-08-31,541870.0,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,Queens,-7921.333333


In [236]:
sample = new_frame.loc[new_frame['ntaname'] == "Whitestone"]
sample

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,new
151,Whitestone,Queens,2,716150.5,714378.5,712513.5,712687.0,712395.5,714074.5,716273.0,719226.5,722598.0,725292.0,1772.0


In [198]:
curr_region = sample.iloc[0, 0]
curr_region

'Whitestone'

In [238]:
sample.iloc[:, len(sample.columns)-1:len(sample.columns)]

,new
151,1772.0


In [200]:
sample['new'] = 0

sample

C:\Users\Sena\.conda\envs\prog\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,new
151,Whitestone,Queens,2,716150.5,714378.5,712513.5,712687.0,712395.5,714074.5,716273.0,719226.5,722598.0,725292.0,0


,new
151,0


In [173]:
sample['new'] = sample["2020-01-31"] - sample["2020-02-29"]

C:\Users\Sena\.conda\envs\prog\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [174]:
sample

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,new
151,Whitestone,Queens,2,716150.5,714378.5,712513.5,712687.0,712395.5,714074.5,716273.0,719226.5,722598.0,725292.0,1772.0


In [177]:
first = dates_frame.loc[dates_frame['date'] == '2020-02-29']

In [178]:
second = dates_frame.loc[dates_frame['date'] == '2020-01-31']

In [186]:
temp = new_frame

In [187]:
new_frame['new'] = new_frame["2020-01-31"] - new_frame["2020-02-29"]

In [188]:
new_frame

,ntaname,boro_name,count,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,new
0,Borough Park,Brooklyn,1,800107.000000,794346.0,792962.000000,792549.000000,792589.000000,791801.000000,790573.000000,792553.0,794147.000000,792330.0,5761.000000
1,Murray Hill,Queens,1,384721.000000,384396.0,383443.000000,383618.000000,383362.000000,383960.000000,384397.000000,384940.0,386321.000000,388206.0,325.000000
2,East Elmhurst,Queens,1,663453.000000,665648.0,667851.000000,669964.000000,671708.000000,672445.000000,672120.000000,672802.0,676621.000000,681728.0,-2195.000000
3,Erasmus,Brooklyn,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
4,Hollis,Queens,1,554907.000000,555797.0,556189.000000,557034.000000,557172.000000,558160.000000,560169.000000,562641.0,566751.000000,568297.0,-890.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,park-cemetery-etc-Brooklyn,Brooklyn,0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
191,Richmond Hill,Queens,1,571840.000000,572333.0,572168.000000,572098.000000,571145.000000,572393.000000,574503.000000,577717.0,581482.000000,583228.0,-493.000000
192,Jamaica,Queens,1,439332.000000,440256.0,441169.000000,442618.000000,443684.000000,445890.000000,448625.000000,451519.0,455501.000000,458605.0,-924.000000
193,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,Queens,3,531565.333333,533002.0,533948.666667,536517.666667,537704.666667,539185.333333,539979.666667,541870.0,544999.666667,548942.0,-1436.666667


In [185]:
temp.reset_index(drop=True)

NameError: name 'temp' is not defined

In [206]:
dates[0]

'2020-01-31'

In [240]:
temp = new_frame[dates[3]]

0      792549.000000
1      383618.000000
2      669964.000000
3           0.000000
4      557034.000000
           ...      
190         0.000000
191    572098.000000
192    442618.000000
193    536517.666667
194         0.000000
Name: 2020-04-30, Length: 195, dtype: float64

In [254]:
temp = new_frame.loc[new_frame[dates[3]] > 0]

temp[dates[3]].min()

165928.0